# 1. O modelo de dados do Go

## 1.2. Um baralho gopher

### 1.2.1. Estrutura `Carta`

`Carta` é um estrutura (`struct`) que representa uma carta de baralho.

In [1]:
type Carta struct {
	Valor string
	Naipe string
}

O "sete belo" é conhecido como "beer card" nos EUA:

In [2]:
var cartaDaCerveja = Carta{Valor: "7", Naipe: "ouros"}
%%
fmt.Println(cartaDaCerveja.Valor, cartaDaCerveja.Naipe)
fmt.Println(cartaDaCerveja)

7 ouros
{7 ouros}


Agora podemos vincular o método `String` à estrutura:

In [3]:
func (carta Carta) String() string {
	return fmt.Sprintf("%s de %s", carta.Valor, carta.Naipe)
}

In [4]:
%%
fmt.Println(cartaDaCerveja)

7 de ouros


### 1.2.2. Estrutura Baralho

Baralho é uma classe que representa um "baralho francês", o tipo mais comum no Brasil, com 52 cartas em 4 naipes de 13 cartas.

In [5]:
type Baralho struct {
	cartas []Carta
}

func Naipes() []string {
	return []string{"copas", "ouros", "paus", "espadas"}
}

func Valores() []string {
	return []string{"2", "3", "4", "5", "6", "7", "8", "9", "10", "J", "K", "Q", "A"}
}

func NovoBaralho() *Baralho {
	cartas := make([]Carta, 0, 52)
	for _, naipe := range Naipes() {
		for _, valor := range Valores() {
			cartas = append(cartas, Carta{Naipe: naipe, Valor: valor})
		}
	}
	return &Baralho{cartas}
}

func (baralho Baralho) Len() int {
	return len(baralho.cartas)
}

func (baralho Baralho) NaPosição(i int) Carta {
	return baralho.cartas[i]
}

func (baralho Baralho) Fatiar(de, até int) []Carta {
	return baralho.cartas[de:até]
}

func (baralho Baralho) Contém(carta Carta) bool{
    return slices.Contains(baralho.cartas, carta)
}

In [6]:
var (
 baralho = NovoBaralho()
)
%%
fmt.Println(baralho.Len())
fmt.Println(baralho.NaPosição(3))
fmt.Println(baralho.Fatiar(0, 3))

fmt.Println(baralho.Contém(Carta{"Q", "espadas"}))

52
5 de copas
[2 de copas 3 de copas 4 de copas]
true


Notas: 
* Em Go não é possivel definir um método para fazer len(baralho), como o `__len__` do Python. Por isso foi criado o `Len`.
* Os métodos `NaPosição` e `Fatiar` são substitutos ao `__getitem__`
* O método `Contém` é substituto ao `in`.

#### Exercício

Crie uma carta que não existe, e verifique que o `Contém` devolve `false`

In [7]:
%%
fmt.Println(baralho.Contém(Carta{"H", "espadas"}))

false


O laço for sabe lidar com sequências:

In [8]:
func (baralho Baralho) Cartas()iter.Seq[Carta]{
    return slices.Values(baralho.cartas)
}

Nota: O nome ser `Cartas` é para ser idiomático. https://go.dev/doc/effective_go#Getters

In [9]:
%%
for carta := range baralho.Cartas(){
    fmt.Println(carta)
}

2 de copas
3 de copas
4 de copas
5 de copas
6 de copas
7 de copas
8 de copas
9 de copas
10 de copas
J de copas
K de copas
Q de copas
A de copas
2 de ouros
3 de ouros
4 de ouros
5 de ouros
6 de ouros
7 de ouros
8 de ouros
9 de ouros
10 de ouros
J de ouros
K de ouros
Q de ouros
A de ouros
2 de paus
3 de paus
4 de paus
5 de paus
6 de paus
7 de paus
8 de paus
9 de paus
10 de paus
J de paus
K de paus
Q de paus
A de paus
2 de espadas
3 de espadas
4 de espadas
5 de espadas
6 de espadas
7 de espadas
8 de espadas
9 de espadas
10 de espadas
J de espadas
K de espadas
Q de espadas
A de espadas


Para escolher um elemento na sequência de forma aleatória:

In [10]:
%%
posiçãoAleatória := rand.IntN(baralho.Len())
fmt.Println(baralho.NaPosição(posiçãoAleatória))

3 de paus


Mas o embaralhamento não funciona. Resolver isso será um exercício logo mais:

In [11]:
// rand.Shuffle(baralho.Len(), func(i, j int) {
//		tmp := baralho.NaPosição(i)
//		baralho.ColocarNaPosição(i, baralho.NaPosição(j))
//		baralho.ColocarNaPosição(j, tmp)
//	})

In [12]:
%%
ordenado := slices.SortedFunc(baralho.Cartas(),  func(a, b Carta) int {
    if n := strings.Compare(a.Valor, b.Valor); n!= 0 {
        return n
    }
    return strings.Compare(a.Naipe, b.Naipe)
})
for _, carta := range ordenado {
    fmt.Println(carta)
}

10 de copas
10 de espadas
10 de ouros
10 de paus
2 de copas
2 de espadas
2 de ouros
2 de paus
3 de copas
3 de espadas
3 de ouros
3 de paus
4 de copas
4 de espadas
4 de ouros
4 de paus
5 de copas
5 de espadas
5 de ouros
5 de paus
6 de copas
6 de espadas
6 de ouros
6 de paus
7 de copas
7 de espadas
7 de ouros
7 de paus
8 de copas
8 de espadas
8 de ouros
8 de paus
9 de copas
9 de espadas
9 de ouros
9 de paus
A de copas
A de espadas
A de ouros
A de paus
J de copas
J de espadas
J de ouros
J de paus
K de copas
K de espadas
K de ouros
K de paus
Q de copas
Q de espadas
Q de ouros
Q de paus


Podemos criar uma função que estabelece um critério de ordenação melhor:

In [13]:
func ordenaçãoEspadasMaior(a, b Carta) int {
	valoresDoNaipe := map[string]int{"espadas": 3, "copas": 2, "ouros": 1, "paus": 0}
	valores := Valores()

	indiceDoValorA := slices.Index(valores, a.Valor)
	valorA := indiceDoValorA*len(valoresDoNaipe) + valoresDoNaipe[a.Naipe]
    
    indiceDoValorB := slices.Index(valores, b.Valor)
	valorB := indiceDoValorB*len(valoresDoNaipe) + valoresDoNaipe[b.Naipe]

	return cmp.Compare(valorA, valorB)
}

In [14]:
%%
for _, carta := range slices.SortedFunc(baralho.Cartas(),  ordenaçãoEspadasMaior) {
    fmt.Println(carta)
}

2 de paus
2 de ouros
2 de copas
2 de espadas
3 de paus
3 de ouros
3 de copas
3 de espadas
4 de paus
4 de ouros
4 de copas
4 de espadas
5 de paus
5 de ouros
5 de copas
5 de espadas
6 de paus
6 de ouros
6 de copas
6 de espadas
7 de paus
7 de ouros
7 de copas
7 de espadas
8 de paus
8 de ouros
8 de copas
8 de espadas
9 de paus
9 de ouros
9 de copas
9 de espadas
10 de paus
10 de ouros
10 de copas
10 de espadas
J de paus
J de ouros
J de copas
J de espadas
K de paus
K de ouros
K de copas
K de espadas
Q de paus
Q de ouros
Q de copas
Q de espadas
A de paus
A de ouros
A de copas
A de espadas


#### Exercício

Defina uma nova ordem que classifique as cartas primeiro por naipe e depois por valor, de forma que todos os paus venham primeiro, seguidos por todos os ouros, etc.

In [15]:
func ordenaçãoNaipesPrimeiro(a, b Carta) int {
	valoresDoNaipe := map[string]int{"espadas": 3, "copas": 2, "ouros": 1, "paus": 0}
	valores := Valores()

	indiceDoValorA := slices.Index(valores, a.Valor)
	valorA := valoresDoNaipe[a.Naipe]*len(valores) + indiceDoValorA

    indiceDoValorB := slices.Index(valores, b.Valor)
	valorB := valoresDoNaipe[b.Naipe]*len(valores) + indiceDoValorB

	return cmp.Compare(valorA, valorB)
}

In [16]:
%%
for _, carta := range slices.SortedFunc(baralho.Cartas(),  ordenaçãoNaipesPrimeiro) {
    fmt.Println(carta)
}

2 de paus
3 de paus
4 de paus
5 de paus
6 de paus
7 de paus
8 de paus
9 de paus
10 de paus
J de paus
K de paus
Q de paus
A de paus
2 de ouros
3 de ouros
4 de ouros
5 de ouros
6 de ouros
7 de ouros
8 de ouros
9 de ouros
10 de ouros
J de ouros
K de ouros
Q de ouros
A de ouros
2 de copas
3 de copas
4 de copas
5 de copas
6 de copas
7 de copas
8 de copas
9 de copas
10 de copas
J de copas
K de copas
Q de copas
A de copas
2 de espadas
3 de espadas
4 de espadas
5 de espadas
6 de espadas
7 de espadas
8 de espadas
9 de espadas
10 de espadas
J de espadas
K de espadas
Q de espadas
A de espadas


#### Exercício

Escreva um método chamado `ColocarNaPosição` que pega um baralho, um índice e uma carta e atribui a carta ao baralho na posição dada.

"Então tenter embaralhar usando `rand.Shuffle`."

In [17]:
func (baralho *Baralho) ColocarNaPosição(i int, carta Carta) {
	baralho.cartas[i] = carta
}

In [18]:
%%
rand.Shuffle(baralho.Len(), func(i, j int) {
    tmp := baralho.NaPosição(i)
    baralho.ColocarNaPosição(i, baralho.NaPosição(j))
    baralho.ColocarNaPosição(j, tmp)
})
for carta := range baralho.Cartas() {
    fmt.Println(carta)
}

K de copas
6 de paus
Q de paus
4 de ouros
5 de paus
8 de copas
10 de paus
9 de ouros
6 de espadas
8 de espadas
Q de ouros
10 de espadas
5 de ouros
9 de espadas
J de ouros
3 de copas
5 de espadas
5 de copas
3 de espadas
8 de paus
2 de paus
4 de espadas
J de espadas
9 de paus
A de espadas
6 de copas
2 de ouros
9 de copas
K de ouros
2 de copas
8 de ouros
10 de copas
4 de copas
3 de ouros
3 de paus
2 de espadas
A de copas
4 de paus
J de paus
7 de espadas
7 de ouros
10 de ouros
A de paus
Q de copas
K de paus
K de espadas
7 de paus
J de copas
A de ouros
Q de espadas
6 de ouros
7 de copas


#### Exercício bônus

A operação de fatiamento `x[a:b]` normalmente devolve uma instância da mesma classe de `x`.

Hoje a função Fatiar devolve uma lista de cartas `[]Card` ao invés de um `Baralho`. Como modificar isto?

Será necessário alterar `Baralho` para fazer isso acontecer? Como?

In [19]:
func BaralhoAPartirDe(cartas []Carta) *Baralho {
	return &Baralho{cartas}
}

func (baralho Baralho) Fatiar(de, até int) *Baralho {
	return BaralhoAPartirDe(baralho.cartas[de:até])
}

%%
// Embaralhando antes de distribuir cartas
rand.Shuffle(baralho.Len(), func(i, j int) {
    tmp := baralho.NaPosição(i)
    baralho.ColocarNaPosição(i, baralho.NaPosição(j))
    baralho.ColocarNaPosição(j, tmp)
})

mão := baralho.Fatiar(0, 5)
fmt.Printf("Tipo: %T\n", mão)
for carta := range mão.Cartas() {
    fmt.Println(carta)
}

Tipo: *main.Baralho
9 de ouros
10 de ouros
3 de espadas
5 de copas
K de espadas
